# CS Schedule Parser

### Connect to Firebase

In [2]:
from firebase import firebase
firebase = firebase.FirebaseApplication("https://ulink-f40aa.firebaseio.com/")

### Connect to UMass Course page with Selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

url = "https://www.cics.umass.edu/content/fall-18-course-schedule"
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.get(url)

### Hand it over to BeautifulSoup

In [39]:
from bs4 import BeautifulSoup
soup_level1 = BeautifulSoup(driver.page_source, 'lxml')

import re
line = "10:23 AM 04:25 PM"

time = re.compile('([0-9]{2}:[0-9]{2}( AM| PM))')

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

from collections import defaultdict
courses = defaultdict(dict)

next_is_detail = False # used in for loop to identify when a row of course detail is coming
course_number = ""
course_name = ""
for row in soup_level1.find_all('tr'): # find all rows of the table
    if next_is_detail: # this will fill in detail for courses
        cells = row.find_all('td')
        courses[course_number]["days"] = cells[2].text
        
        time_matches = re.findall(time,cells[3].text)
        if len(time_matches) == 2:
            courses[course_number]["start_time"] = time_matches[0][0]
            courses[course_number]["end_time"] = time_matches[1][0]
        else:
            courses[course_number]["start_time"] = "TBA"
            courses[course_number]["end_time"] = "TBA"
        courses[course_number]["location"] = cells[4].text
        next_is_detail = False
        continue
        
        
    if len(row.find_all('td')) == 2:  # course rows should only have two entries
        count = 0
        course_number = ""
        course_name = ""
        for cell in row.find_all('td'): # cells are in td tags which have a tags holding info we want
            link = cell.find_all('a')[0]
            if count == 0:  # first entry should be name of course
                if not is_number(link.text):  # if it isnt a number, skip this row; it isnt a class
                    break
                else:
                    course_number = link.text
                    count += 1
            if count == 1:
                course_name = link.text
        if course_name is not "":
            courses[course_number]["course_name"] = course_name # default dict solves KeyError problem
            next_is_detail = True

courses = dict(courses)  
print(courses)

{'105': {'course_name': 'Computer Literacy', 'days': 'TueThu', 'start_time': '11:30 AM', 'end_time': '12:45 PM', 'location': 'HasbAd 20'}, '119': {'course_name': 'Introduction to Programming', 'days': 'MonWedFri', 'start_time': '01:25 PM', 'end_time': '02:15 PM', 'location': 'Goes 20'}, '120': {'course_name': 'Introduction to Problem Solving with the Internet', 'days': 'MonWedFri', 'start_time': '11:15 AM', 'end_time': '12:05 PM', 'location': 'ILCS331'}, '121': {'course_name': 'Introduction to Problem Solving with Computers', 'days': 'Tue', 'start_time': '11:30 AM', 'end_time': '12:45 PM', 'location': 'ILCN151'}, '186': {'course_name': 'Using Data Structures', 'days': 'TueThu', 'start_time': '04:00 PM', 'end_time': '05:15 PM', 'location': 'ILCS331'}, '187': {'course_name': 'Programming with Data Structures', 'days': 'TueThu', 'start_time': '04:00 PM', 'end_time': '05:15 PM', 'location': 'Thmpsn 104'}, '220': {'course_name': 'Programming Methodology', 'days': 'TueThu', 'start_time': '02

### Define function for pushing to Firebase

In [40]:
def pushToFirebase(new_info):
    update = firebase.patch('/course_info', new_info)

### Push data to Firebase

In [41]:
pushToFirebase(courses)